In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import os
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.modeling import build_model
import torch
import json

In [3]:
setup_logger()

# register_coco_instances("nuclei_dataset", {}, "nucleus_cocoformat.json", "./train")
# register_coco_instances("nuclei_dataset", {}, "test.json", "./train")
register_coco_instances("nuclei_dataset", {}, "nucleus_cocoformat_poly2.json", "./train")
register_coco_instances("nuclei_dataset_val", {}, "nucleus_cocoformat_poly2.json", "./val")
metadata = MetadataCatalog.get("nuclei_dataset")
dataset_dicts = DatasetCatalog.get("nuclei_dataset")

[12/06 23:16:49 d2.data.datasets.coco]: Loading nucleus_cocoformat_poly2.json takes 1.48 seconds.
[12/06 23:16:49 d2.data.datasets.coco]: Loaded 24 images in COCO format from nucleus_cocoformat_poly2.json


In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

from detectron2.structures import BoxMode
def get_balloon_dicts(img_dir):
    print(img_dir)
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for _, anno in annos.items():
            assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["train", "val"]:
    DatasetCatalog.register("balloon_" + d, lambda d=d: get_balloon_dicts("balloon/" + d))
    MetadataCatalog.get("balloon_" + d).set(thing_classes=["balloon"])
balloon_metadata = MetadataCatalog.get("balloon_train")

In [ ]:
dataset_dicts = get_balloon_dicts("balloon/train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    # cv2_imshow(out.get_image()[:, :, ::-1])
    # cv2.imshow("out", out.get_image()[:, :, ::-1])
    cv2.imwrite('output.jpg', out.get_image()[:, :, ::-1])

In [102]:
cfg = get_cfg()
# cfg.merge_from_file("./model/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
# cfg.OUTPUT_DIR = "./output"
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml')
# cfg.MODEL.WEIGHTS = os.path.join('model', "model_final_Cascade.pkl")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# if you have pre-trained weight.
cfg.DATASETS.TRAIN = ("nuclei_dataset",)
# cfg.DATASETS.TRAIN = ("balloon_train",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 0
cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025
# cfg.SOLVER.BASE_LR = 0.00005
cfg.SOLVER.BASE_LR = 0.000001
# 4999 iterations seems good enough, but you can certainly train longer
cfg.SOLVER.MAX_ITER = 100
# faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 8
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 classes (data, fig, hazelnut)
# cfg.MODEL.DEVICE ='cuda'
# cfg.INPUT.MASK_FORMAT = 'bitmask'

import torch
print(torch.cuda.is_available())

# model = build_model(cfg)
# model = model.to('cuda')

True


In [ ]:
'''
model = build_model(cfg)

from detectron2.data import transforms as T
from detectron2.data import DatasetMapper   # the default mapper
from detectron2.data import build_detection_train_loader

dataloader = build_detection_train_loader(cfg,
   mapper=DatasetMapper(cfg, is_train=True, augmentations=[
      T.Resize((1000, 1000))
   ]), num_workers=0)

from detectron2.engine import SimpleTrainer
from detectron2.engine import HookBase
from detectron2.solver import get_default_optimizer_params


optimizer = torch.optim.Adam(get_default_optimizer_params(model),
               lr=1e-4)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

class CheckerHook(HookBase):
  def after_step(self):
    if self.trainer.iter % 100 == 0:
      print(f"Iteration {self.trainer.iter} complete")



trainer = SimpleTrainer(model, data_loader=dataloader, optimizer = optimizer) 
trainer.register_hooks([CheckerHook()])


trainer.train(start_iter=0, max_iter=100)
'''

In [103]:
print(cfg.OUTPUT_DIR
)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)  # build output folder
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

./output
[12/07 17:11:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
 

In [91]:
# Look at training curves in tensorboard:
# !kill 20112
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir output

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 20112), started 1 day, 1:29:22 ago. (Use '!kill 20112' to kill it.)

In [5]:
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    # cv2_imshow(out.get_image()[:, :, ::-1])
    # cv2.imshow("out", out.get_image()[:, :, ::-1])
    cv2.imwrite('output_poly.jpg', out.get_image()[:, :, ::-1])

In [104]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [105]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("nuclei_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "nuclei_dataset_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[12/07 17:16:04 d2.data.datasets.coco]: Loaded 24 images in COCO format from nucleus_cocoformat_poly2.json
[12/07 17:16:04 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/07 17:16:04 d2.data.common]: Serializing 24 elements to byte tensors and concatenating them all ...
[12/07 17:16:04 d2.data.common]: Serialized dataset takes 22.23 MiB
[12/07 17:16:04 d2.evaluation.evaluator]: Start inference on 24 batches
[12/07 17:16:09 d2.evaluation.evaluator]: Inference done 11/24. Dataloading: 0.0993 s/iter. Inference: 0.1106 s/iter. Eval: 0.2222 s/iter. Total: 0.4321 s/iter. ETA=0:00:05
[12/07 17:16:15 d2.evaluation.evaluator]: Inference done 23/24. Dataloading: 0.1175 s/iter. Inference: 0.1102 s/iter. Eval: 0.2203 s/iter. Total: 0.4480 s/iter. ETA=0:00:00
[12/07 17:16:15 d2.evaluation.evaluator]: Total inference time: 0:00:08.568224 (0.450959 s / iter per device, on 1 devices)
